In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions

from data.toy_regression import create_linear_data, ground_truth_linear_function
from core.preprocessing import preprocess_create_x_train_test
from core.network_utils import build_model

In [4]:
input_shape = [1]
layer_units = [2]
layer_activations = ["linear"]


In [5]:
n_train = 20
_x_train, y_train = create_linear_data(n_train, m=3, b=1, sigma=1)
x_train, _x_test, x_test = preprocess_create_x_train_test(_x_train)
y_test = ground_truth_linear_function(_x_test)

In [6]:
w = np.array([1, 0]).reshape(1, -1)
b = np.zeros((2)) 
b[1] = 12

In [48]:
def target_log_prob_fn(wb):
    print(wb)
    _w = np.expand_dims(wb[0, 0])#tf.reshape(tf.stack((w, tf.constant(0., dtype='float32')), axis=0), (1, -1))
    _b = wb[0, 1]
    print(_w.shape)
    net = tf.matmul(_x_train, _w) + _b
    y_pred, y_unconstrained_std = tf.unstack(net, axis=1)
    prediction = tfp.distributions.Normal(loc=y_pred, scale=1e-6 + tf.math.softplus(0.05 * y_unconstrained_std))
    return tf.reduce_sum(prediction.log_prob(y_train))

In [49]:
wb = np.array([[1, 1], [0, 0]]).reshape((1,) + (2, 2))

In [50]:
wb[0, 0]

array([1, 1])

In [51]:
# model = build_model(input_shape, layer_units, layer_activations)
# def target_log_prob_fn(x):
#     print(x)
#     x = tf.reshape(x, (1, 1))
#     ls = model(x)
#     l = ls[0, 0]
#     s = ls[0, 1]
#     return tf.reduce_sum(tfd.Normal(loc=l, scale=1.).log_prob(np.ones(5)))

# Initialize the HMC transition kernel.
initial_state = np.array([[1, 1], [0, 0]], dtype='float32')
initial_state = initial_state.reshape((1, ) + initial_state.shape)
num_results = int(1e2)
num_burnin_steps = int(30)
step_size = 1.
num_leapfrog_steps = 2
step_size_adapter = "dual_averaging"
sampler = "hmc"

step_size_adapter = {
    "simple": tfp.mcmc.SimpleStepSizeAdaptation,
    "dual_averaging": tfp.mcmc.DualAveragingStepSizeAdaptation,
}[step_size_adapter]
if sampler == "nuts":
    kernel = tfp.mcmc.NoUTurnSampler(target_log_prob_fn, step_size=step_size)
    adaptive_kernel = step_size_adapter(
        kernel,
        num_adaptation_steps=int(num_burnin_steps * 0.8),
        step_size_setter_fn=lambda pkr, new_step_size: pkr._replace(
            step_size=new_step_size
        ),
        step_size_getter_fn=lambda pkr: pkr.step_size,
        log_accept_prob_getter_fn=lambda pkr: pkr.log_accept_ratio,
    )
elif sampler == "hmc":
    kernel = tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn,
        step_size=step_size,
        num_leapfrog_steps=num_leapfrog_steps,
    )
    adaptive_kernel = step_size_adapter(
        kernel, num_adaptation_steps=int(num_burnin_steps * 0.8),
    )

# Run the chain (with burn-in).
#@tf.function
def run_chain():
  # Run the chain (with burn-in).
    samples, is_accepted = tfp.mcmc.sample_chain(
      num_results=num_results,
      num_burnin_steps=num_burnin_steps,
      current_state=initial_state,
      kernel=adaptive_kernel,
      trace_fn=lambda _, pkr: pkr.inner_results.is_accepted)

    return samples, is_accepted

samples, is_accepted = run_chain()
sample_mean = tf.reduce_mean(samples)
sample_stddev = tf.math.reduce_std(samples)
is_accepted = tf.reduce_mean(tf.cast(is_accepted, dtype=tf.float32))

print('mean:{:.4f}  stddev:{:.4f}  acceptance:{:.4f}'.format(
    sample_mean.numpy(), sample_stddev.numpy(), is_accepted.numpy()))

tf.Tensor(
[[[1. 1.]
  [0. 0.]]], shape=(1, 2, 2), dtype=float32)
(2,)


InvalidArgumentError: In[1] is not a matrix. Instead it has shape [2] [Op:MatMul] name: mcmc_sample_chain/dual_averaging_step_size_adaptation___init__/_bootstrap_results/mh_bootstrap_results/hmc_kernel_bootstrap_results/maybe_call_fn_and_grads/value_and_gradients/MatMul/

In [52]:
samples

<tf.Tensor: shape=(100, 1, 2), dtype=float32, numpy=
array([[[0.15731606, 1.7696476 ]],

       [[0.13462949, 1.8311614 ]],

       [[0.10783669, 1.8606288 ]],

       [[0.10783669, 1.8606288 ]],

       [[0.10610261, 1.8693013 ]],

       [[0.12161877, 1.7428343 ]],

       [[0.08214192, 1.7536923 ]],

       [[0.13221863, 1.6996726 ]],

       [[0.11793502, 1.6624676 ]],

       [[0.13173181, 1.6960788 ]],

       [[0.11650054, 1.7869112 ]],

       [[0.10637537, 1.7492055 ]],

       [[0.12507032, 1.7743533 ]],

       [[0.08862688, 1.7211844 ]],

       [[0.15290585, 1.658581  ]],

       [[0.11831407, 1.6270393 ]],

       [[0.15279323, 1.6760348 ]],

       [[0.10023018, 1.7907156 ]],

       [[0.12068367, 1.8609116 ]],

       [[0.1027743 , 1.9342159 ]],

       [[0.06719038, 1.9506385 ]],

       [[0.10886997, 1.895876  ]],

       [[0.09780123, 1.9167316 ]],

       [[0.08841945, 1.7928467 ]],

       [[0.11299814, 1.7922671 ]],

       [[0.10827324, 1.8181018 ]],

       [[0.

In [20]:
model((np.arange(10)-5).reshape(-1, 1))

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[-3.6336331 , -5.605876  ],
       [-2.9069066 , -4.4847007 ],
       [-2.18018   , -3.3635254 ],
       [-1.4534533 , -2.2423503 ],
       [-0.72672665, -1.1211752 ],
       [ 0.        ,  0.        ],
       [ 0.72672665,  1.1211752 ],
       [ 1.4534533 ,  2.2423503 ],
       [ 2.18018   ,  3.3635254 ],
       [ 2.9069066 ,  4.4847007 ]], dtype=float32)>

In [160]:
samples

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([ 2.3257124 , -0.20577645,  0.19978791,  0.03424105, -0.08349824,
        1.5081794 ,  1.4159038 ,  2.0308464 , -0.10871029,  0.28580248,
        0.902448  ,  0.07596786,  1.2128446 ,  0.887585  ,  1.4048815 ,
        1.5153879 , -1.7769922 ,  0.6979221 ,  0.09650894,  0.20094928,
        1.7372818 ,  0.72853154,  0.3511223 ,  0.8152547 ,  3.8389397 ,
        1.5429881 ,  1.8295449 ,  1.4188477 ,  3.4186873 ,  0.35176575,
        1.4801888 ,  0.9883277 ,  0.7906201 ,  0.15862834, -1.1441915 ,
        1.921804  ,  1.2293398 ,  1.7564956 ,  1.8990531 , -0.4522009 ,
       -0.3013208 ,  1.9032283 ,  1.4277234 , -0.84536034,  2.0777595 ,
       -1.0363597 , -0.37081444, -0.56208473,  1.3032736 ,  1.5694382 ,
        0.920076  ,  1.8546269 , -0.24779195, -0.24779195,  1.9183695 ,
        0.20211214,  0.36986756,  1.3416731 ,  1.0556343 ,  1.5287949 ,
        1.6589696 ,  0.8491319 ,  0.07071872,  0.7900215 , -0.07740676,
       -0.077406